<a href="https://colab.research.google.com/github/midhuu8989/Python_login/blob/newbranch/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RJ VOICE CLONING::


# New Section

In [1]:
!pip install -U scipy
!git clone https://github.com/neonbjb/tortoise-tts.git
%cd tortoise-tts
!pip install -r requirements.txt
!python setup.py install

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 17.8 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.14.1 which is incompatible.
Cloning into 'tortoise-tts'...
remote: Enumerating objects: 1987, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 1987 (delta 0), reused 2 (delta 0), pack-reused 1982 (from 1)
Receiving objects: 100% (1987/1987), 54.19 MiB | 32.62 MiB/s, done.
Resolving deltas: 100% (911/911), done.
/content/tortoise-tts
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/1

running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https

In [3]:
!pip install gradio
import os
import gradio as gr
import torchaudio
import time
from datetime import datetime
from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

VOICE_OPTIONS = [
    "random",  # special option for random voice
    "custom_voice",  # special option for custom voice
    "disabled",  # special option for disabled voice
]

def inference(text, emotion, prompt, voice, mic_audio, voice_b, voice_c, preset, seed):
    if voice != "custom_voice":
        voices = [voice]
    else:
        voices = []

    if voice_b != "disabled":
        voices.append(voice_b)
    if voice_c != "disabled":
        voices.append(voice_c)

    if emotion != "None/Custom":
        text = f"[I am really {emotion.lower()},] {text}"
    elif prompt.strip() != "":
        text = f"[{prompt},] {text}"

    c = None
    if voice == "custom_voice":
        if mic_audio is None:
            raise gr.Error("Please provide audio from mic when choosing custom voice")
        c = load_audio(mic_audio, 22050)

    if len(voices) == 1 or len(voices) == 0:
        if voice == "custom_voice":
            voice_samples, conditioning_latents = [c], None
        else:
            voice_samples, conditioning_latents = load_voice(voice)
    else:
        voice_samples, conditioning_latents = load_voices(voices)
        if voice == "custom_voice":
            voice_samples.extend([c])

    sample_voice = voice_samples[0] if len(voice_samples) else None

    start_time = time.time()
    gen, _ = tts.tts_with_preset(
        text,
        voice_samples=voice_samples,
        conditioning_latents=conditioning_latents,
        preset=preset,
        use_deterministic_seed=seed,
        return_deterministic_state=True,
        k=3,
    )

    with open("Tortoise_TTS_Runs.log", "a") as f:
        f.write(
            f"{datetime.now()} | Voice: {','.join(voices)} | Text: {text} | Quality: {preset} | Time Taken (s): {time.time()-start_time} | Seed: {seed}\n"
        )

    return (
        (22050, sample_voice.squeeze().cpu().numpy()),
        (24000, gen[0].squeeze().cpu().numpy()),
        (24000, gen[1].squeeze().cpu().numpy()),
        (24000, gen[2].squeeze().cpu().numpy()),
    )

def main():
    text = gr.Textbox(lines=4, label="Text:")
    emotion = gr.Radio(
        ["None/Custom", "Happy", "Sad", "Angry", "Disgusted", "Arrogant"],
        value="None/Custom",
        label="Select emotion:",
        type="value",
    )
    prompt = gr.Textbox(lines=1, label="Enter prompt if [Custom] emotion:")
    preset = gr.Radio(
        ["ultra_fast", "fast", "standard", "high_quality"],
        value="fast",
        label="Preset mode (determines quality with tradeoff over speed):",
        type="value",
    )
    voice = gr.Dropdown(
        os.listdir(os.path.join("tortoise", "voices")) + VOICE_OPTIONS,
        value="angie",  # Default voice
        label="Select voice:",
        type="value",
    )
    mic_audio = gr.Audio(
        label="Record voice (when selected custom_voice):",
        type="filepath"  # Changed from 'source' to just 'type'
    )
    voice_b = gr.Dropdown(
        os.listdir(os.path.join("tortoise", "voices")) + VOICE_OPTIONS,
        value="disabled",
        label="(Optional) Select second voice:",
        type="value",
    )
    voice_c = gr.Dropdown(
        os.listdir(os.path.join("tortoise", "voices")) + VOICE_OPTIONS,
        value="disabled",
        label="(Optional) Select third voice:",
        type="value",
    )
    seed = gr.Number(value=0, precision=0, label="Seed (for reproducibility):")

    selected_voice = gr.Audio(label="Sample of selected voice (first):")
    output_audio_1 = gr.Audio(label="Output [Candidate 1]:")
    output_audio_2 = gr.Audio(label="Output [Candidate 2]:")
    output_audio_3 = gr.Audio(label="Output [Candidate 3]:")

    interface = gr.Interface(
        fn=inference,
        inputs=[
            text,
            emotion,
            prompt,
            voice,
            mic_audio,
            voice_b,
            voice_c,
            preset,
            seed,
        ],
        outputs=[selected_voice, output_audio_1, output_audio_2, output_audio_3],
    )
    interface.launch(share=True)

if __name__ == "__main__":
    tts = TextToSpeech()

    with open("Tortoise_TTS_Runs.log", "a") as f:
        f.write(
            f"\n\n-------------------------Tortoise TTS Logs, {datetime.now()}-------------------------\n"
        )

    main()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.4/447.4 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.10/dist-packages/rotary_embedding_torch/rotary_embedding_torch.py:29: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/usr/local/lib/python3.10/dist-packages/rotary_embedding_torch/rotary_embedding_torch.py:189: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings ta

config.json:   0%|          | 0.00/2.11k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:460: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location="cp

...

clvp2.pth:   0%|          | 0.00/976M [00:00<?, ?B/s]

/content/tortoise-tts/tortoise/api.py:233: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.clvp.load_state_dict(torch.load(get_model_path('clvp2.pth', models_dir)))
/usr/

vocoder.pth:   0%|          | 0.00/391M [00:00<?, ?B/s]

/content/tortoise-tts/tortoise/api.py:237: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.vocoder.load_state_dict(torch.load(get_model_path('vocoder.pth', models_dir), m

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8e7f70fa3f384d467f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:

!pip install gradio
%cd tortoise-tts
import os
import gradio as gr
import torchaudio
import time
from datetime import datetime
from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

VOICE_OPTIONS = [
    "random",  # special option for random voice
    "custom_voice",  # special option for custom voice
    "disabled",  # special option for disabled voice
]

def inference(text, emotion, prompt, voice, mic_audio, voice_b, voice_c, preset, seed):
    if voice != "custom_voice":
        voices = [voice]
    else:
        voices = []

    if voice_b != "disabled":
        voices.append(voice_b)
    if voice_c != "disabled":
        voices.append(voice_c)

    if emotion != "None/Custom":
        text = f"[I am really {emotion.lower()},] {text}"
    elif prompt.strip() != "":
        text = f"[{prompt},] {text}"

    c = None
    if voice == "custom_voice":
        if mic_audio is None:
            raise gr.Error("Please provide audio from mic when choosing custom voice")
        c = load_audio(mic_audio, 22050)

    if len(voices) == 1 or len(voices) == 0:
        if voice == "custom_voice":
            voice_samples, conditioning_latents = [c], None
        else:
            voice_samples, conditioning_latents = load_voice(voice)
    else:
        voice_samples, conditioning_latents = load_voices(voices)
        if voice == "custom_voice":
            voice_samples.extend([c])

    sample_voice = voice_samples[0] if len(voice_samples) else None

    start_time = time.time()
    gen, _ = tts.tts_with_preset(
        text,
        voice_samples=voice_samples,
        conditioning_latents=conditioning_latents,
        preset=preset,
        use_deterministic_seed=seed,
        return_deterministic_state=True,
        k=3,
    )

    with open("Tortoise_TTS_Runs.log", "a") as f:
        f.write(
            f"{datetime.now()} | Voice: {','.join(voices)} | Text: {text} | Quality: {preset} | Time Taken (s): {time.time()-start_time} | Seed: {seed}\n"
        )

    return (
        (22050, sample_voice.squeeze().cpu().numpy()),
        (24000, gen[0].squeeze().cpu().numpy()),
        (24000, gen[1].squeeze().cpu().numpy()),
        (24000, gen[2].squeeze().cpu().numpy()),
    )

def main():
    # Custom HTML for the heading
    title = "<h1 style='text-align: center; color: orange; font-weight: bold;'>RJ VOICE CLONING</h1>"

    # Interface components
    text = gr.Textbox(lines=4, label="Text:")
    emotion = gr.Radio(
        ["None/Custom", "Happy", "Sad", "Angry", "Disgusted", "Arrogant"],
        value="None/Custom",
        label="Select emotion:",
        type="value",
    )
    prompt = gr.Textbox(lines=1, label="Enter prompt if [Custom] emotion:")
    preset = gr.Radio(
        ["ultra_fast", "fast", "standard", "high_quality"],
        value="fast",
        label="Preset mode (determines quality with tradeoff over speed):",
        type="value",
    )
    voice = gr.Dropdown(
        os.listdir(os.path.join("tortoise", "voices")) + VOICE_OPTIONS,
        value="angie",  # Default voice
        label="Select voice:",
        type="value",
    )
    mic_audio = gr.Audio(
        label="Record voice (when selected custom_voice):",
        type="filepath"
    )
    voice_b = gr.Dropdown(
        os.listdir(os.path.join("tortoise", "voices")) + VOICE_OPTIONS,
        value="disabled",
        label="(Optional) Select second voice:",
        type="value",
    )
    voice_c = gr.Dropdown(
        os.listdir(os.path.join("tortoise", "voices")) + VOICE_OPTIONS,
        value="disabled",
        label="(Optional) Select third voice:",
        type="value",
    )
    seed = gr.Number(value=0, precision=0, label="Seed (for reproducibility):")

    selected_voice = gr.Audio(label="Sample of selected voice (first):")
    output_audio_1 = gr.Audio(label="Output [Candidate 1]:")
    output_audio_2 = gr.Audio(label="Output [Candidate 2]:")
    output_audio_3 = gr.Audio(label="Output [Candidate 3]:")

    # Define the Gradio interface
    interface = gr.Interface(
        fn=inference,
        inputs=[text, emotion, prompt, voice, mic_audio, voice_b, voice_c, preset, seed],
        outputs=[selected_voice, output_audio_1, output_audio_2, output_audio_3],
        theme="default",
        description=title
    )

    # Launch the interface
    interface.launch(share=True)

if __name__ == "__main__":
    tts = TextToSpeech()

    with open("Tortoise_TTS_Runs.log", "a") as f:
        f.write(
            f"\n\n-------------------------Tortoise TTS Logs, {datetime.now()}-------------------------\n"
        )

    main()


[Errno 2] No such file or directory: 'tortoise-tts'
/content/tortoise-tts


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:460: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serializa